In [1]:
import pandas as pd
df = pd.read_csv("C:/Users/wongzn/Desktop/df_dec.csv")

## NLTK

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [3]:
def nltk_ner(doc):
    return {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(doc))) if hasattr(chunk,'label')}

In [4]:
import time

In [5]:
st=time.time()

In [6]:
nltk_ent = df['Article Content'].apply(lambda x: nltk_ner(x))

In [7]:
time.time()-st

786.0410177707672

In [8]:
df_nltk_ent = pd.DataFrame(columns=['Entity','Final'])

for i in range(len(nltk_ent)):
    df = pd.DataFrame(nltk_ent[i], columns = ['Entity','Final'])
    df_nltk_ent = df_nltk_ent.append(df)

In [9]:
df_nltk_ent.to_csv("C:/Users/wongzn/Desktop/nltk_ent.csv")

In [10]:
df_nltk_ent.Final.value_counts()

ORGANIZATION    24092
PERSON          21451
GPE             14484
FACILITY          514
LOCATION          294
GSP               203
Name: Final, dtype: int64

In [11]:
df_nltk_ent[df_nltk_ent.Final.str.contains('ORGANIZATION')].head()

,Entity,Final
0,CNB,ORGANIZATION
1,Maxwell Road,ORGANIZATION
2,International,ORGANIZATION
8,Orchard Towers,ORGANIZATION
9,Central Narcotics Bureau,ORGANIZATION


## Spacy

In [12]:
import spacy
sp_lg = spacy.load('en_core_web_lg')

In [13]:
def spacy_ner(doc):
    return {(ent.text.strip(),ent.label_) for ent in sp_lg(doc).ents}

In [16]:
df = pd.read_csv("C:/Users/wongzn/Desktop/df_dec.csv")

In [17]:
st = time.time()

In [18]:
spacy_ent = df['Article Content'].apply(lambda x: spacy_ner(x))

In [19]:
time.time()-st

672.3008227348328

In [20]:
df_spacy_ent = pd.DataFrame(columns=['Entity','Final'])

for i in range(len(spacy_ent)):
    df = pd.DataFrame(spacy_ent[i], columns = ['Entity','Final'])
    df_spacy_ent = df_spacy_ent.append(df)

In [21]:
df_spacy_ent.to_csv("C:/Users/wongzn/Desktop/spacy_ent.csv")

In [22]:
df_spacy_ent.Final.value_counts()

DATE           28560
ORG            21974
PERSON         16513
CARDINAL       13482
GPE            10000
MONEY           4918
FAC             3190
NORP            2687
ORDINAL         2432
TIME            2373
LOC             1695
QUANTITY        1151
PRODUCT          750
EVENT            729
LAW              561
WORK_OF_ART      535
LANGUAGE         251
PERCENT           15
Name: Final, dtype: int64

In [23]:
df_spacy_ent[df_spacy_ent.Final.str.contains('FAC')].Entity.value_counts().head()

Changi Airport             56
Orchard Road               37
MRT                        30
Jewel Changi Airport       25
the Shangri-La Dialogue    24
Name: Entity, dtype: int64

## Polyglot

In [24]:
from polyglot.text import Text

In [28]:
df = pd.read_csv("C:/Users/wongzn/Desktop/df_dec.csv")

In [29]:
def polyglot_ner(document):
    return {(' '.join(entity), entity.tag.split('-')[-1]) for entity in Text(document).entities}

In [30]:
st=time.time()

In [31]:
polyglot_ent = df['Article Content'].apply(lambda x: polyglot_ner(x))

In [32]:
time.time()-st

455.7363827228546

In [33]:
df_polyglot_ent = pd.DataFrame(columns=['Entity','Final'])

for i in range(len(polyglot_ent)):
    df = pd.DataFrame(polyglot_ent[i], columns = ['Entity','Final'])
    df_polyglot_ent = df_polyglot_ent.append(df)

In [34]:
df_polyglot_ent.to_csv('C:/Users/wongzn/Desktop/polyglot_ent.csv')

In [35]:
df_polyglot_ent.Final.value_counts()

ORG    19552
PER    18524
LOC    14007
Name: Final, dtype: int64

In [36]:
df_polyglot_ent[df_polyglot_ent.Final.str.contains('ORG')].Entity.value_counts().head(30)

Straits Times                       455
The Straits Times                   337
Facebook                            250
Singapore                           205
Singapore Civil Defence Force       189
Government                          180
Police                              160
National University of Singapore    145
State                               144
National Environment Agency         124
Transport Authority                 114
NUS                                 108
Senior                              104
Health                              104
NEA                                 100
Transport                            98
Home Affairs                         84
Industry                             80
Nanyang Technological University     80
Singapore Armed Forces               79
LTA                                  79
Manpower                             78
Singapore Police Force               72
Singapore Press Holdings             68
Singapore General Hospital           67


## Gate (Not Working)

In [219]:
df = pd.read_csv("C:/Users/wongzn/Desktop/df_dec.csv")

In [ ]:
import requests
url = "https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer"
headers = {'Content-Type': 'text/plain'}
response = requests.post(url, data=example_document, headers=headers).json()

import json
print(json.dumps(response, indent=2))

In [ ]:
def gate_ner(sentence):
  import requests
  return [(sentence[entity["indices"][0]:entity["indices"][1]] + f" ({entity['gender']})",entity_type) if entity_type == "Person" and "gender" in entity else (sentence[entity["indices"][0]:entity["indices"][1]],entity_type)  for entity_type,entities in requests.post("https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer", data=sentence, headers={'Content-Type': 'text/plain'}).json()["entities"].items() for entity in entities]

In [ ]:
gate_ent = df['Article Content'].apply(lambda x: gate_ner(x))

In [ ]:
gate_ner(example_document)

## Flair

In [ ]:
import flair
from flair.models import SequenceTagger
flair_12class = SequenceTagger.load('ner-ontonotes-fast')
flair_4class = SequenceTagger.load('ner')
from segtok.segmenter import split_single
from flair.data import Sentence

In [ ]:
#try splitting

def flair_ner(document,model):
    from flair.data import Sentence
    sentences = split_single(document)
    s = [Sentence(sent,use_tokenizer=True) for sent in sentences]
    model.predict(s)
    entities = s.to_dict(tag_type='ner')
    return [(entity["text"], entity["type"]) for entity in entities["entities"]]

In [ ]:
def flair_ner(document,model):
    from flair.data import Sentence
    s= Sentence(document)
    model.predict(s)
    entities = s.to_dict(tag_type='ner')
    return [(entity["text"], entity["type"]) for entity in entities["entities"]]

In [ ]:
flair_ner(example_document, flair_4class)

In [ ]:
flair_ner(example_document, flair_12class)

In [ ]:
import time

In [ ]:
## flair sentence split

In [ ]:
df = pd.read_csv("C:/Users/wongzn/Desktop/df_dec.csv")

In [ ]:
st = time.time()

In [ ]:
#try splitting
def flair_ner(document,model):
    final = []
    from flair.data import Sentence
    sentences = split_single(document)
    for i in range(len(sentences)):
        s = Sentence(sentences[i],use_tokenizer=True)
        model.predict(s)
        entities = s.to_dict(tag_type='ner')
        j = [(entity["text"], entity["type"]) for entity in entities["entities"]]
        final.append(j)
    return final

In [ ]:
#flair_ent_4_ss = df['Article Content'].apply(lambda x: flair_ner(x, flair_4class))

In [9]:
print(time.time()-st)

118788.11805033684


In [12]:
a = [item for x in flair_ent_4_ss for item in x]

In [14]:
flair_ent = [item for x in a for item in x]

In [17]:
backup = flair_ent_4_ss

In [25]:
df_flair_ent = pd.DataFrame(flair_ent, columns=['Entity','Final'])

In [27]:
df_flair_ent.to_csv('C:/Users/wongzn/Desktop/flair_ent.csv')

In [63]:
df_flair_ent.Final.value_counts()

ORG     32503
LOC     24552
PER     23950
MISC    10595
Name: Final, dtype: int64

In [ ]:
df_flair_ent[df_flair_ent.Final.str.contains('MISC')].Entity.value_counts().head(30)

# Ensemble Model

In [74]:
df_flair = pd.read_csv("C:/Users/wongzn/Desktop/flair_ent.csv")
df_pg =  pd.read_csv("C:/Users/wongzn/Desktop/polyglot_ent.csv")
df_spacy =  pd.read_csv("C:/Users/wongzn/Desktop/spacy_ent.csv")
df_nltk =  pd.read_csv("C:/Users/wongzn/Desktop/nltk_ent.csv")

In [39]:
df_flair = df_flair[df_flair.Final.str.contains('ORG')]
df_pg = df_pg[df_pg.Final.str.contains('ORG')]
df_spacy = df_spacy[df_spacy.Final.str.contains('ORG')]
df_nltk = df_nltk[df_nltk.Final.str.contains('ORGANIZATION')]

In [64]:
df_flair = df_flair[df_flair.Final.str.contains('LOC')]
df_pg = df_pg[df_pg.Final.str.contains('LOC')]
df_spacy = df_spacy[df_spacy.Final.str.contains('LOC')]
df_nltk = df_nltk[df_nltk.Final.str.contains('GPE')]

In [75]:
df_flair = df_flair[df_flair.Final.str.contains('PER')]
df_pg = df_pg[df_pg.Final.str.contains('PER')]
df_spacy = df_spacy[df_spacy.Final.str.contains('PERSON')]
df_nltk = df_nltk[df_nltk.Final.str.contains('PERSON')]

In [76]:
df_flair = df_flair[['Entity']]
df_pg = df_pg[['Entity']]
df_spacy = df_spacy[['Entity']]
df_nltk = df_nltk[['Entity']]

In [77]:
df_flair.reset_index(inplace=True)
df_flair.rename(columns={'Entity':'flair_ent'},inplace=True)
df_pg.reset_index(inplace=True)
df_pg.rename(columns={'Entity': 'pg_ent'},inplace=True)
df_spacy.reset_index(inplace=True)
df_spacy.rename(columns={'Entity':'spacy_ent'},inplace=True)
df_nltk.reset_index(inplace=True)
df_nltk.rename(columns={'Entity':'nltk_ent'},inplace=True)

In [78]:
df = pd.concat([df_flair,df_pg,df_spacy,df_nltk],axis=1)

In [79]:
df.drop(['index'],axis=1,inplace=True)

In [80]:
a = set(df['flair_ent'])
b = set(df['spacy_ent'])
c = set(df['pg_ent'])
d = set(df['nltk_ent'])

In [81]:
one = a & b
two = a & c
three = a & d

In [82]:
#ensemble_location = one.union(two.union(three))
ensemble_person = one.union(two.union(three))
#ensemble_org = one.union(two.union(three))

In [86]:
ensemble_location

{'Telangana',
 'Busan',
 'South China Seas',
 'Port of Singapore',
 'Marsiling Lane',
 'Chin',
 'Tokyo Haneda',
 'Hong Kong',
 'Enterprise Road',
 'Nauru',
 'Riau Islands',
 'Sheraton Towers',
 'Casablanca',
 'Pine Grove',
 'Mullumbimby',
 'Mistri Road',
 'South-East Asia',
 'Serangoon',
 'Chile',
 'Central Catchment Nature Reserve',
 'Central Java',
 'Dawson Road',
 'Pasir Gudang',
 'SEOUL',
 'Act East',
 'Stockholm',
 'British Virgin Islands',
 'McAllen',
 'Chongqing',
 'Seletar',
 'Transport Hub West',
 'Dairy Farm',
 'Arizona',
 'Mackenzie Road',
 'Port Blair',
 'Selegie',
 'Cassia Crescent',
 'Matrix',
 'Tanjung Resang',
 'RIYADH',
 'NParks',
 'Upper Bukit Timah',
 'Tan',
 'Batam',
 'South East',
 'Scotland',
 'Hill Street',
 'Gojek',
 'Chas',
 'Express',
 'New Zealand',
 'Hollywood',
 'East-West Line',
 'Moscow',
 'Nanchang',
 'Halifax Road',
 'Brazil',
 'Indramayu',
 'Coney Island',
 'Asia Pacific',
 'Bedok North',
 'Ho Chi Minh City',
 'North Coast',
 'Boat Quay',
 'Sepang',
 '

In [83]:
ensemble_person

{'Ng Jun Chong',
 'Ong Keng Yong',
 'Muhammad Raziq Mohamad',
 'Zkert M. Rushdi',
 'Tony Yeo',
 'Faizahhanimi Ahmad',
 'Mike Chew Jun Yong',
 'Pannir Selvam Pranthaman',
 'Makoto Nakagawa',
 'Ruhunadevi Joshua',
 'Theophileous',
 'Pasir Gudang',
 'Kulamani Ganesan',
 'Cheng Li Hui',
 'JASON QUAH Madam Assirah',
 'Steven Heng',
 'Grace Quek Xin Hui',
 'Charles Quek',
 'Daniel Goh',
 'Siow',
 'Hashim Abdul Rahman',
 'Yew',
 'Mohamed Rafeeq Mohamed Yusoof',
 'Edwin Tong',
 'Sgt Riduan',
 'Jaishankar',
 'Slappy Cakes',
 'Ms Quah',
 'Maimunah Awang',
 'Tan Huay Cheem',
 'Shauna Tan',
 'Elaine Ng',
 'Tan Lian Heng',
 'Jeremy Sun',
 'Ika Zahri Sarkasi',
 'Ai Tong',
 'Asep Ardiansyah',
 'Barry Desker',
 'Yap Thanabadee',
 'Madam Wong Li Li',
 'Ishwarpal Singh Grewal',
 'Caballero',
 'Gabriel Lien Goh',
 'T. Raja Kumar',
 'Madam Ng Bee Leng',
 'Koo Tsai Kee',
 'Brian Batchelor',
 'Low Ming Wah',
 'Nur Khairunnisa Mohd Ridzwan',
 'Geraldine Lam',
 'Bernadette Chirac',
 'Rodrigo Duterte',
 'Ho Kw

In [47]:
ensemble_org

{'Korean International Company',
 'Johnson & Johnson',
 'Kimly',
 'National Development and Manpower',
 'Camokakis',
 'Confinement Angels',
 'National Population Health Survey',
 'SembCorp',
 'SparkCognition',
 'DJ',
 'Asean Economic Community',
 'CWO',
 'Samsung',
 'ISAS',
 'International Transport Forum',
 'Singapore History Gallery',
 'BuildTech Asia',
 'Unity',
 'WSH Council and Enterprise Singapore',
 'URA',
 'Slappy Cakes',
 'ABC Waters',
 'West Coast Town Council',
 'Inclusion Ambassadors',
 'Supreme Court',
 'Science Centre Board',
 'Accounting Standards Council',
 'BeeX',
 'HDB',
 'Koufu',
 'Islamic Theology',
 'Home Team',
 'SANA',
 'US Energy Information Administration',
 'ABB',
 'Tinkering Studio',
 'Singapore FinTech Festival',
 'Short Stay Unit',
 'International SOS',
 'SKH',
 'IBA',
 'American Chamber of Commerce',
 'SPH News Centre Auditorium',
 'SPO',
 'Public Transport Vouchers',
 'The States Times Review',
 'Flyer',
 'Kitchen Catering',
 '7th Marine Regiment',
 'Semb